# HateSpeech Detection in Gujarati Language

## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import string
import csv
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt 
import pickle
import ast

## DataSet

In [2]:
data = pd.read_csv(r'C:\Users\Dhruv.DHRUVPC\Gujarati_NLP.csv')
data.head()

,ID,Post,Labels,Simple_Labels,SubCategory,"Simple_Labels,SubCategory",MainCategory,SubCategory.1
0,1,બંન્ને કેવી ગાળો બોલે .. કોય ને સાથ ના આપતા..આ...,"""Non-Hate"",""Neutral""",Non-Hate,Neutral,"Non-Hate,Neutral","""Non-Hate""","""Neutral"""
1,2,જય જય ગરવી ગુજરાત,"""Non-Hate"",""Positive""",Non-Hate,Positive,"Non-Hate,Positive","""Non-Hate""","""Positive"""
2,3,કચ્છમાં ખંડણી માટે ફાયરિંગ કરીને હત્યા નિપજાવા...,"""Non-Hate"",""Neutral""",Non-Hate,Neutral,"Non-Hate,Neutral","""Non-Hate""","""Neutral"""
3,4,આજે સમગ્ર વિસ્તારમાં ઠંડકનો માહોલ જોવા મળ્યો હ...,"""Non-Hate"",""Positive""",Non-Hate,Positive,"Non-Hate,Positive","""Non-Hate""","""Positive"""
4,5,પાટીદાર અનામત આંદોલનકારીઓએ હવે રાજય સરકાર સામે...,"""Non-Hate"",""Neutral""",Non-Hate,Neutral,"Non-Hate,Neutral","""Non-Hate""","""Neutral"""


In [3]:
type(data['Labels'].iloc[1])

str

In [4]:
data["Post"][1]

'જય જય ગરવી ગુજરાત'

## Text Cleanning - Data Pre-Processing

In [5]:
STOPWORDS = ['અથવા', 'અને', 'અમને', 'અમારું', 'અમે ', 'અહીં', 'આ ', 'આગળ', 'આથી', 'આનું', 'આને', 'આપણને', 'આપણું', 'આપણે',
             'આપી', 'આવી', 'આવે', 'ઉપર', 'ઊંચે', 'ઊભું', 'એ', 'એક','એના','એનાં','એની','એનું','એને','એનો','એમ','એવા','એવાં',
             'એવી','એવું','એવો','ઓછું','અંગે','અંદર','કઈ','કયું','કયો','કરવું','કરતાં','કરી','કરીએ','કરું','કરે','કરેલું', 'કર્યા','કર્યાં','કર્યું',
             'કર્યો','કંઈક','કાંઈ','કે','કેટલું','કેમ','કેવી','કેવું ','કોઈ','કોઈક','કોણ','કોણે','કોને','ક્યારે','ક્યાં','ખૂબ ','ગઈ ','ગયા ','ગયાં'
             ,'ગયું ','ગયો','ઘણું','છ ','છતાં','છીએ', 'છું ','છે','છેક','છો','જ','જાય','જી','જે','જેટલું','જેને','જેમ','જેવી','જેવું','જેવો','જો',
             'જોઈએ','જ્યારે','જ્યાં ','ઝાઝું','તને ','તમને','તમારું' ,'તમે ','તારાથી ','તારામાં','તારું','તું ','તે ','તેઓ ','તેથી ','તેણે ','તેના '
             ,'તેની','તેનું ','તેને','તેમ','તેમનું','તેમને','તેવી','તેવું','તેં','તો ','ત્યારે','ત્યાં','થઈ','થઈએ' ,'થતા ','થતાં','થતી ','થતું','થતો',
             'થયા ','થયાં','થયું ','થયો','થયેલું','થવું ','થાઉં','થાઓ','થાય','થોડું','દરેક','ન','નથી' ,'નહિ' ,'નહીં' ,'નં.' ,'ના' ,'નીચે' ,
             'ને' ,'પછી' ,'પણ' ,'પર' ,'પરંતુ','પહેલાં','પાછળ' ,'પાસે' ,'પોતાનું' ,'પ્રત્યેક' ,'ફક્ત' ,'ફરી' ,'ફરીથી' ,'બધા' ,'બધું','બની' ,
             'બહાર' ,'બહુ' ,'બંને' ,'બાદ' ,'બે' ,'મને' ,'મા' ,'માટે' ,'માત્ર' ,'મારું' ',મૂકવું' ',મૂકી' ',મૂક્યા' ',મૂક્યાં' ,'મૂક્યું' ,'મેં' ,'રહી',
             'રહે' ,'રહેવું','રહ્યા','રહ્યાં','રહ્યો' ,'રીતે' ,'રૂ.','લેતા','લેતું' ,'લેવા','વગેરે' ,'વધુ' ,'શકે' ,'શા' ,'શું' ,'સરખું' ,'સામે' ,'સુધી'
             ,'હતા' ,'હતાં' ,'હતી' ,'હતું' ,'હશે','હશો' ,'હવે' ,'હા' ,'હું','હો' ,'હોઈ' ,'હોઈશ' ,'હોઈશું' ,'હોય','હોવા','+',  'ોરોના', '્ઓલખી', 'ૐ', '૦૪', '૧', '૧માંથી', '૧મૈજર', '૧૦', '૧૦૦', '૧૦૦૦', '૧૦૦૦રુ', '૧૦૦૧૦૦', '૧૦૯', '૧૧૦','%', '&', "''", '(', ')', '+સિકયુરિટ', ';', '<', '>', '``', 'à', 'अकाउंट', 'असली', 'आरंमिहु', 'औरंगजेब', 'काम', 'जयहींद', 'थी', 'ने', 'पर', 'बदले', 'भाई', 'मतलब', 'मालिक','અ', 'मे', 'या', 'वर्ष', 'वाह', 'साथ', '૧૨', '૧૨૦કૅરોર', '૧૩૦', '૧૩૫', '૧૪', '૧૫', '૧૫લાખ', '૧૫૦', '૧૬', '૧૮૮', '૧૯૮૦', '૨', '૨૦', '૨૦૦', '૨૦૧૨', '૨૦૧૩', '૨૦૧૪', '૨૦૧૫', '૨૦૧૭', '૨૦૧૮', '૨૦૧૯', '૨૦૨૨', '૨૦૨૯', '૨૨', '૨૩', '૨૪', '૨૫', '૨૬', '૨૬૯', '૨૭', '૨૭૦', '૨૮૦', '૩', '૩ર્ડ', '૩૦', '૩૦૦', '૩૧', '૩૧ડીસેમ્બર', '૩૩૫', '૩૫', '૩૫૦૦', '૩૫૪૦', '૪', '૪૦', '૪૧', '૪૫', '૪૫૦', '૫', '૫૦', '૫૦૦', '૫૬', '૬', '૬૦', '૬૦૬૫', '૭', '૭૪', '૮થી', '૮૦૦', '૮૫૦', '૯', '૯૦', '૯૦૦', '૯૭', '૯૮', '\u200bપેટની', '\u200bભોસરો', '\u200c', '\u200cઆવા', '\u200cએક', '\u200cએટલ', '\u200cકરતા', '\u200cકરાઈ', '\u200cકરાવે', '\u200cકાઢવાનો', '\u200cટાગોર', '\u200cતમારે', '\u200cતે', '\u200cથાવી', '\u200cનથી', '\u200cને', '\u200cપોતાના', '\u200cબરાબર', 'ધંધ\u200dે', 'ધ',  'નાથી\u200e', '\u200cભાવનગર', '\u200cમોધવારી', '\u200cસાચૂ', '\u200c\u200c', '\u200e', '\u200eઆર્મી', '\u200eપોલીસ', '\u200eફેકો', '‘', '’', '“', '”', '₹', '₹પપ્પુ', '₹૫૦૦૦૦', '℅', '●','છે૧૦૦૦', 'છે\u200e', 'છૈ', 'છેં', 'છેઃ','થ્યા', 'થ\u200cઈ', 'દ',]
            

In [6]:
def stopword_removal(new_post):
    tokens = word_tokenize(new_post)
    stopword_remove_row = [word for word in tokens if word not in STOPWORDS]
    return stopword_remove_row

In [7]:
def emojis_removal(post):
    list1 = []
    for i in post:
        allchars = [str for str in i]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        clean_text = ' '.join([str for str in i.split() if not any(i in str for i in emoji_list)])
        list1.append(clean_text)
    return list1

In [8]:
def puncuation_removal(list):
    list2 = []
    for i in list:
        new_str = re.sub(r'[a-zA-Z0-9\n\',.#@_:…।?/|!$*-]', r'',i)
        list2.append(new_str)
    return list2

In [9]:
post = data["Post"]
new_post = emojis_removal(post)
post1 = puncuation_removal(new_post)


In [10]:
data_x = post1
#data_x =post
data_y = data["Labels"]


In [11]:
len(data_x)

5498

In [12]:
len(data_y)

5498

In [13]:
data_y.head()

0     "Non-Hate","Neutral"
1    "Non-Hate","Positive"
2     "Non-Hate","Neutral"
3    "Non-Hate","Positive"
4     "Non-Hate","Neutral"
Name: Labels, dtype: object

In [14]:
type(data_x)

list

In [15]:
data_dict = { 
    
    "post": data_x,
    "labels": data_y,   
}

In [16]:
type(data_dict)

dict

In [17]:
data_new = pd.DataFrame(data_dict)
data_new.head()

,post,labels
0,બંન્ને કેવી ગાળો બોલે કોય ને સાથ ના આપતાઆપડે ...,"""Non-Hate"",""Neutral"""
1,જય જય ગરવી ગુજરાત,"""Non-Hate"",""Positive"""
2,કચ્છમાં ખંડણી માટે ફાયરિંગ કરીને હત્યા નિપજાવા...,"""Non-Hate"",""Neutral"""
3,આજે સમગ્ર વિસ્તારમાં ઠંડકનો માહોલ જોવા મળ્યો હ...,"""Non-Hate"",""Positive"""
4,પાટીદાર અનામત આંદોલનકારીઓએ હવે રાજય સરકાર સામે...,"""Non-Hate"",""Neutral"""


In [18]:
ast.literal_eval(str(data_new['labels'].iloc[0]))

('Non-Hate', 'Neutral')

In [19]:
data_new['labels'] = data_new['labels'].apply(lambda x:ast.literal_eval(x))


## MultiLabelBinarizer

In [20]:
multilabel = MultiLabelBinarizer()

In [21]:
y = multilabel.fit_transform(data_new['labels'])

In [22]:
y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [23]:
multilabel.classes_

array(['Accusations', 'Hate', 'Neutral', 'Non-Hate', 'Positive',
       'Promoting violence', 'Sexist', 'Swearing'], dtype=object)

In [24]:
pd.DataFrame(y,columns=multilabel.classes_)

,Accusations,Hate,Neutral,Non-Hate,Positive,Promoting violence,Sexist,Swearing
0,0,0,1,1,0,0,0,0
1,0,0,0,1,1,0,0,0
2,0,0,1,1,0,0,0,0
3,0,0,0,1,1,0,0,0
4,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
5493,0,1,0,0,0,1,0,0
5494,0,1,0,0,0,0,0,1
5495,0,1,0,0,0,0,0,1
5496,0,1,0,0,0,1,0,0


In [25]:
tfidf = TfidfVectorizer(analyzer=stopword_removal)

X = tfidf.fit_transform(data_new["post"])
print(X.shape)

(5498, 17469)


In [26]:
print(y.shape)

(5498, 8)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [28]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

In [29]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100


def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print('Jacard score: {}'.format(j_score(y_test, y_pred)))
    print('----')

In [30]:
for classifier in [LinearSVC(C=1.5, penalty = 'l1', dual=False)]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  LinearSVC
Jacard score: 50.75757575757576
----


In [31]:
for classifier in [sgd, lr, svc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)


Clf:  SGDClassifier
Jacard score: 52.037878787878796
----
Clf:  LogisticRegression
Jacard score: 46.39393939393939
----
Clf:  LinearSVC
Jacard score: 51.85606060606061
----


## RealData test

In [32]:
your_input = ['કાર્યકરો ને મારો']

In [33]:
xt = tfidf.transform(your_input)

In [34]:
clf.predict(xt)

array([[0, 1, 0, 0, 0, 1, 0, 0]])

In [35]:
multilabel.inverse_transform(clf.predict(xt))

[('Hate', 'Promoting violence')]